### 강의에서 소개된 파이썬 주요 기능
- matplotlib.pyplot.axvspan: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.axvspan.html
- matplotlib.animation.FuncAnimation: https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.FuncAnimation.html
- matplotlib.colors.Normalize: https://matplotlib.org/stable/api/_as_gen/matplotlib.colors.Normalize.html

### 스프린트 검출

##### (1) 가공 데이터 불러오기

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
from src.plot_utils import draw_pitch

mpl.rcParams['animation.embed_limit'] = 100

In [ ]:
match_id = 1
file = f'data_metrica/Sample_Game_{match_id}/Sample_Game_{match_id}_IntegratedData_Reshaped.csv'
long_traces = pd.read_csv(file, header=0, index_col=0)
long_traces

##### (2) 시간-속력 그래프 애니메이션 시각화

In [ ]:
player_id = 8
player_trace = long_traces[long_traces['player_id'] == player_id]

fig, ax = plt.subplots(figsize=(15, 5))
line, = ax.plot(player_trace['time'], player_trace['speed'], color='grey')

width = 60
ax.axhline(7, color='blue')
ax.axhline(15, color='green')
ax.axhline(20, color='orange')
ax.axhline(25, color='red')
ax.grid()

ax.set_xlabel('Time [s]', fontsize=15)
ax.set_ylabel('Speed [m/s]', fontsize=15)
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(15)

ax.set(xlim=(0, width), ylim=(0, 40))
plt.show()

In [ ]:
def animate(i):
    ax.set_xlim(10 * i, 10 * i + width)
    return line,

frames = int((player_trace['time'].max() - width) // 10 + 1)
anim = animation.FuncAnimation(fig, animate, frames=frames, interval=200, blit=True)
plt.close(fig)
HTML(anim.to_jshtml())

##### (3) 고속 주행 시간 간격을 활용한 선수 스프린트 검출

In [ ]:
high_speed_slices = player_trace[player_trace['speed'] >= 25]
high_speed_slices[100:120]

In [ ]:
high_speed_slices['time_diff'] = high_speed_slices['time'].diff()
high_speed_slices[high_speed_slices['time_diff'] > 0.5]

In [ ]:
high_speed_slices['sprint_id'] = (high_speed_slices['time_diff'] > 0.5).astype(int).cumsum() + 1
high_speed_slices

In [ ]:
player_sprint_records = pd.DataFrame(index=high_speed_slices['sprint_id'].unique())

grouped = high_speed_slices.groupby('sprint_id')
player_sprint_records[['period', 'start_frame', 'start_time']] = grouped[['period', 'frame', 'time']].first()
player_sprint_records[['end_frame', 'end_time']] = grouped[['frame', 'time']].last()

player_sprint_records['start_frame'] = player_sprint_records['start_frame'] - 1
player_sprint_records['start_time'] = (player_sprint_records['start_time'] - 0.04).round(2)
player_sprint_records['duration'] = player_sprint_records['end_time'] - player_sprint_records['start_time']

player_sprint_records['distance'] = grouped['distance'].sum()
player_sprint_records['max_speed'] = grouped['speed'].max()

player_sprint_records

##### (4) 경기 전체 스프린트 검출

In [ ]:
sprint_records_list = []

for player_id in tqdm(long_traces['player_id'].unique()):
    player_trace = long_traces[long_traces['player_id'] == player_id]
    high_speed_slices = player_trace[player_trace['speed'] >= 25]
    time_diffs = high_speed_slices['time'].diff()
    high_speed_slices['sprint_id'] = (time_diffs > 0.5).astype(int).cumsum() + 1

    player_sprint_records = pd.DataFrame(index=high_speed_slices['sprint_id'].unique())

    grouped = high_speed_slices.groupby('sprint_id')
    player_sprint_records[['period', 'start_frame', 'start_time']] = grouped[['period', 'frame', 'time']].first()
    player_sprint_records[['end_frame', 'end_time']] = grouped[['frame', 'time']].last()

    player_sprint_records['start_frame'] = player_sprint_records['start_frame'] - 1
    player_sprint_records['start_time'] = (player_sprint_records['start_time'] - 0.04).round(2)
    player_sprint_records['duration'] = player_sprint_records['end_time'] - player_sprint_records['start_time']

    player_sprint_records['distance'] = grouped['distance'].sum()
    player_sprint_records['max_speed'] = grouped['speed'].max()

    player_sprint_records['team'] = player_trace['team'].iloc[0]
    player_sprint_records['player_id'] = player_id
    sprint_records_list.append(player_sprint_records[player_sprint_records['duration'] >= 0.5])

sprint_records = pd.concat(sprint_records_list)
cols = sprint_records.columns.tolist()[-2:] + sprint_records.columns.tolist()[:-2]
sprint_records = sprint_records[cols].sort_values(['period', 'start_time'])

sprint_records.index = np.arange(len(sprint_records)) + 1
sprint_records.index.name = 'sprint_id'
sprint_records

##### (5) 선수별 스프린트 횟수 집계

In [ ]:
durations = long_traces[['player_id', 'x']].dropna().groupby('player_id').count() * 0.04
distances = long_traces.groupby('player_id')['distance'].sum()
durations.columns = ['duration']
stats = pd.concat([durations, distances], axis=1)
stats['dist_1min'] = stats['distance'] / stats['duration'] * 60

speed_bins = [0, 7, 15, 20, 25, 50]
speed_cats = pd.cut(long_traces['speed'], speed_bins, right=False, labels=np.arange(1, 6))
distances_by_speed = long_traces.pivot_table('distance', index='player_id', columns=speed_cats, aggfunc='sum')
distances_by_speed.columns = [f'zone{i}_dist' for i in distances_by_speed.columns]
stats = pd.concat([stats, distances_by_speed], axis=1)

grouped = long_traces.groupby('player_id')
stats['max_speed'] = grouped['speed'].max()
stats['mean_x'] = grouped['x'].mean()
stats['mean_y'] = grouped['y'].mean()
stats['team'] = grouped['team'].first()
stats = stats.reset_index().set_index(['team', 'player_id']).round(2)
stats

In [ ]:
stats['sprint_count'] = sprint_records.groupby(['team', 'player_id'])['start_time'].count()
stats['sprint_count'] = stats['sprint_count'].fillna(0).astype(int)
stats

### 스프린트 시각화

##### (1) 선수별 시간-속력 그래프에 스프린트 시점 표시

In [ ]:
player_id = 8
player_trace = long_traces[long_traces['player_id'] == player_id].set_index('frame')
player_sprint_records = sprint_records[sprint_records['player_id'] == player_id]
player_sprint_records

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))

width = 60
ax.set(xlim=(0, width), ylim=(0, 40))
ax.axhline(7, color='blue')
ax.axhline(15, color='green')
ax.axhline(20, color='orange')
ax.axhline(25, color='red')
ax.grid()

ax.set_xlabel('Time [s]', fontsize=15)
ax.set_ylabel('Speed [m/s]', fontsize=15)
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(15)

line, = ax.plot(player_trace['time'], player_trace['speed'], color='grey')
for i in player_sprint_records.index:
    start_time = player_sprint_records.at[i, 'start_time']
    end_time = player_sprint_records.at[i, 'end_time']
    ax.axvspan(start_time, end_time, color='red', alpha=0.5)

def animate(i):
    ax.set_xlim(10 * i, 10 * i + width)
    return line,

frames = int((player_trace['time'].max() - width) // 10 + 1)
anim = animation.FuncAnimation(fig, animate, frames=frames, interval=200, blit=True)
plt.close(fig)
HTML(anim.to_jshtml())

##### (2) 선수별 전체 스프린트 경로 시각화

In [ ]:
cmap = mpl.cm.get_cmap('cool')
norm = mpl.colors.Normalize(vmin=25, vmax=35)
norm(30), cmap(norm(30))

In [ ]:
fig, ax = draw_pitch('white', 'black', size_x=10.4*1.2, size_y=6.8)
cmap = mpl.cm.get_cmap('cool')
norm = mpl.colors.Normalize(vmin=25, vmax=35)

for i, record in player_sprint_records.iterrows():
    period = record['period']
    start_frame = record['start_frame']
    end_frame = record['end_frame']
    sprint_trace = player_trace.loc[start_frame:end_frame]

    x = sprint_trace['x'].values
    y = sprint_trace['y'].values
    color = cmap(norm(record['max_speed']))
    ax.plot(x, y, c=color)
    plt.arrow(x[-2], y[-2], x[-1] - x[-2], y[-1] - y[-2], head_width=1.5, head_length=2, ec=color, fc=color)

sm = mpl.cm.ScalarMappable(cmap=cmap, norm=norm)
cbar = plt.colorbar(sm, ticks=[25, 30, 35])
cbar.ax.tick_params(labelsize=15)
cbar.set_label(label='Max Speed [km/h]', size=15)

ax.set_title(f'Sprint Trajectories for Player{player_id}', fontdict={'size': 20})
plt.show()

##### (3) 형태 변환 이전 데이터에 스프린트 시점 정보 결합

In [ ]:
file = f'data_metrica/Sample_Game_{match_id}/Sample_Game_{match_id}_IntegratedData.csv'
wide_traces = pd.read_csv(file, header=0, index_col=0)
wide_traces

In [ ]:
sprint_records[:10]

In [ ]:
players = [c[:-2] for c in wide_traces.columns if c.endswith('_x') and not c.startswith('ball')]
for p in players:
    wide_traces[f'{p}_sprint'] = 0

for sprint_id, record in sprint_records.iterrows():
    sprinter = f"{record['team'][0]}{record['player_id']:02d}"
    start_frame = record['start_frame']
    end_frame = record['end_frame']
    wide_traces.loc[start_frame:end_frame, f'{sprinter}_sprint'] = sprint_id

wide_traces

In [ ]:
sprint_cols = [c for c in wide_traces.columns if c.endswith('_sprint')]
wide_traces[sprint_cols].loc[1265:1285]

In [ ]:
wide_traces.to_csv(f'data_metrica/Sample_Game_{match_id}/Sample_Game_{match_id}_IntegratedData.csv')

##### (4) 스프린트 장면 애니메이션 시각화

In [ ]:
def plot_snapshot(
    data, figax=None, team_colors=('r', 'b'),
    annotate_players=True, annotate_events=True, show_velocities=True, show_sprints=False
):
    if figax is None:
        fig, ax = draw_pitch(pitch='white', line='black')
    else:
        fig, ax = figax

    figobjs = []
    for team, color in zip(['H', 'A'], team_colors):
        team_players = [c[:-2] for c in data.keys() if c.startswith(team) and c.endswith('_x')]
        team_x_cols = [f'{p}_x' for p in team_players]
        team_y_cols = [f'{p}_y' for p in team_players]

        widths = pd.Series(0, index=team_players)
        if show_sprints:
            team_sprint_cols = [f'{p}_sprint' for p in team_players]
            widths = pd.Series(data[team_sprint_cols].clip(0, 1).values, index=team_players) * 4

        obj = ax.scatter(
            data[team_x_cols], data[team_y_cols], s=100, c=color,
            edgecolors='lime', linewidth=widths
        )
        figobjs.append(obj)

        if show_velocities:
            team_vx_cols = [f'{p}_vx' for p in team_players]
            team_vy_cols = [f'{p}_vy' for p in team_players]
            obj = ax.quiver(
                data[team_x_cols].astype(float), data[team_y_cols].astype(float),
                data[team_vx_cols].astype(float), data[team_vy_cols].astype(float),
                color=color, scale=8, scale_units='inches', width=0.002, alpha=0.7
            )
            figobjs.append(obj)
        
        if annotate_players:
            for x, y in zip(team_x_cols, team_y_cols):
                if not (np.isnan(data[x]) or np.isnan(data[y])):
                    obj = ax.text(data[x] + 0.5, data[y] + 0.5, int(x[1:3]), fontsize=13, color=color)
                    figobjs.append(obj)

    time_text = f"{int(data['time'] // 60):02d}:{data['time'] % 60:05.2f}"
    if annotate_events:
        if not pd.isnull(data['event_subtype']):
            event_text = f"{data['event_subtype']} by {data['event_player']}" 
        else:
            event_text = ''
        figobjs.append(ax.text(51, 69, time_text, fontsize=15, ha='right', va='bottom'))
        figobjs.append(ax.text(52, 69, '-', fontsize=15, ha='center', va='bottom'))
        figobjs.append(ax.text(53, 69, event_text, fontsize=15, ha='left', va='bottom'))
    else:
        figobjs.append(ax.text(52, 69, time_text, fontsize=15, ha='center', va='bottom'))
        
    obj = ax.scatter(data['ball_x'], data['ball_y'], s=80, color='w', edgecolors='k')
    figobjs.append(obj)
    
    return fig, ax, figobjs

In [ ]:
frame = wide_traces[wide_traces['H08_sprint'] > 0].index[0]
fig, ax, figobjs = plot_snapshot(wide_traces.loc[frame], show_sprints=True)
plt.show()

In [ ]:
def save_clip(
    clip_traces, fname='test', fps=25, figax=None, team_colors=('r', 'b'),
    annotate_players=True, annotate_events=True, show_velocities=True, show_sprints=False,
):
    metadata = dict(title='Tracking Data', artist='Matplotlib', comment='Metrica tracking data clip')
    writer = animation.FFMpegWriter(fps=fps, metadata=metadata)

    if not os.path.exists('match_clips'):
        os.makedirs('match_clips')
    file = f'match_clips/{fname}.mp4'

    if figax is None:
        fig, ax = draw_pitch(pitch='white', line='black')
    else:
        fig, ax = figax
    fig.set_tight_layout(True)

    with writer.saving(fig, file, dpi=100):
        for i in clip_traces.index:
            frame_data = clip_traces.loc[i]
            fig, ax, figobjs = plot_snapshot(
                frame_data, (fig, ax), team_colors,
                annotate_players, annotate_events, show_velocities, show_sprints
            )
            writer.grab_frame()

            for obj in figobjs:
                obj.remove()

    plt.clf()
    plt.close(fig)

In [ ]:
start_frame = 1000
end_frame = 1500
show_sprints = True

fname_tokens = [f'game{match_id}', str(start_frame), str(end_frame)]
if show_sprints:
    fname_tokens.append('sprints')
fname = '_'.join(fname_tokens)

clip_traces = wide_traces.loc[start_frame:end_frame]
save_clip(clip_traces, fname=f'{fname}.mp4', show_sprints=show_sprints)